Nome: Francisco Jerônimo da Silva Júnior <br>
Matrícula: 433399

In [2]:
import pandas as pd # processamento de dados
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score # divisão dos dados
from sklearn.preprocessing import StandardScaler, MinMaxScaler # escalonadores
from sklearn.metrics import f1_score # métrica de avaliação
from sklearn.pipeline import Pipeline, make_pipeline # pipelines 
from sklearn.neighbors import KNeighborsClassifier # modelo de classificação

### Avalie o desempenho do algoritmo kNN para o dataset "Pima Indians Diabetes Database (link alternativo no Kaggle)". Separe os dados em treino (80%) e teste (20%).

### 1. Use validação holdout para obter o melhor valor de k para o kNN com e sem escalonamento de features. Para isso, divida o conjunto de treino (80%) entre treino_holdout (70% do conjunto de treino) e validacao_holdout (30% do conjunto de treino). Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação (validacao_holdout). Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

### Leitura do dataset

In [3]:
df = pd.read_csv('diabetes.csv')

### Descrição do dataset

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Separação dos dados em treino e teste

In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)

### Divisão  do conjunto de treino (80%) entre treino_holdout (70% do conjunto de treino) e validacao_holdout (30% do conjunto de treino)

In [7]:
X_train_holdout, X_test_holdout, y_train_holdout, y_test_holdout =  \
train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=42)

### Grid-Search Manual com KNN

In [8]:
results = {'scaler': [], 'k' : [], 'f1_score' : []}

for k in range(2, 26):
    pipe_st = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(k))])
    pipe_st.fit(X_train_holdout, y_train_holdout)
    y_pred = pipe_st.predict(X_test_holdout)
    results['scaler'].append('standard scaler')
    results['k'].append(k)
    results['f1_score'].append(f1_score(y_test_holdout, y_pred))

for k in range(2, 26):
    pipe_mn = Pipeline([('min-max', MinMaxScaler()), ('knn', KNeighborsClassifier(k))])
    pipe_mn.fit(X_train_holdout, y_train_holdout)
    y_pred = pipe_mn.predict(X_test_holdout)
    results['scaler'].append('standard scaler')
    results['k'].append(k)
    results['f1_score'].append(f1_score(y_test_holdout, y_pred))

for k in range(2, 26):
    pipe = Pipeline([('knn', KNeighborsClassifier(k))])
    pipe.fit(X_train_holdout, y_train_holdout)
    y_pred = pipe.predict(X_test_holdout)
    results['scaler'].append('standard scaler')
    results['k'].append(k)
    results['f1_score'].append(f1_score(y_test_holdout, y_pred))

In [9]:
results = pd.DataFrame(results)

results.sort_values(by='f1_score', ascending=False)

,scaler,k,f1_score
69,standard scaler,23,0.635514
57,standard scaler,11,0.633333
61,standard scaler,15,0.632479
29,standard scaler,7,0.625000
53,standard scaler,7,0.622951
...,...,...,...
24,standard scaler,2,0.500000
4,standard scaler,6,0.490196
0,standard scaler,2,0.471910
48,standard scaler,2,0.454545


### Treinando com o melhor modelo usando os dados de treino (80%)

In [10]:
pipeline = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_neighbors=23))])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
f1_score(y_test, y_pred)

0.5806451612903226

### 2. Use validação cruzada com k-Fold=5 para obter o melhor valor de k para o kNN com e sem escalonamento de features. Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação. Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

### Dicas: (1) use a função cross_val_score do scikit learn com cv=5. (2) use o Pipeline do scikit learn para realizar o escalonamento de features. (3) Não use nessa questão o GridSearchCV do scikit learn.

In [11]:
results = {'scaler': [], 'k' : [], 'f1_score_mean' : []}

for k in range(2, 26):
    clf = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=k))
    score = cross_val_score(clf, X_train_holdout, y_train_holdout, cv=5, scoring='f1')
    results['scaler'].append('Standardization')
    results['k'].append(k)
    results['f1_score_mean'].append(score.mean())

for k in range(2, 26):
    clf = make_pipeline(MinMaxScaler(), KNeighborsClassifier(n_neighbors=k))
    score = cross_val_score(clf, X_train_holdout, y_train_holdout, cv=5, scoring='f1')
    results['scaler'].append('Min-Max')
    results['k'].append(k)
    results['f1_score_mean'].append(score.mean())                             

for k in range(2, 26):
    clf = make_pipeline(KNeighborsClassifier(n_neighbors=k))
    score = cross_val_score(clf, X_train_holdout, y_train_holdout, cv=5, scoring='f1')
    results['scaler'].append('no scaler')
    results['k'].append(k)
    results['f1_score_mean'].append(score.mean())

In [12]:
res = pd.DataFrame(results)
res.nlargest(10, 'f1_score_mean')

,scaler,k,f1_score_mean
29,Min-Max,7,0.569131
57,no scaler,11,0.568819
65,no scaler,19,0.566303
63,no scaler,17,0.562264
64,no scaler,18,0.560700
55,no scaler,9,0.559528
59,no scaler,13,0.559000
19,Standardization,21,0.557843
17,Standardization,19,0.555828
51,no scaler,5,0.552059


### Treinando com o melhor modelo usando os dados de treino (80%)

In [13]:
pipeline = make_pipeline(MinMaxScaler(), KNeighborsClassifier(n_neighbors=7))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
f1_score(y_test, y_pred)

0.5436893203883496

### 3. Responda a questão 1 só que agora usando o GridSearchCV do scikit learn. Dica: Nesta questão você não deve usar o cross_val_score do scikit learn.

### Aplicação do Standard Scaler

In [14]:
s = StandardScaler()
X_train_holdout_sta = s.fit_transform(X_train_holdout)
X_test_holdout_sta = s.transform(X_train_holdout)

### Aplicação do Min-Max Scaler

In [15]:
m = MinMaxScaler()
X_train_holdout_mm = m.fit_transform(X_train_holdout)
X_test_holdout_mm = m.transform(X_test_holdout)

### Definição das variações dos hyperparâmetros

In [16]:
params = [ {'n_neighbors' : list(range(2, 26)), 'weights' : ['uniform', 'distance']}]

### Instanciando o KNN

In [17]:
knn = KNeighborsClassifier()

### Criação das grades

In [18]:
grid_minmax = GridSearchCV(knn, params, cv=5, scoring='f1') # vai ser usado com dados normalizados (minmax)
grid_stan = GridSearchCV(knn, params, cv=5, scoring='f1') # vai ser usado com dados padronizados (standard)
grid_search = GridSearchCV(knn, params, cv=5, scoring='f1') # usado com dados sem serem escalonados

In [19]:
grid_search.fit(X_train_holdout, y_train_holdout)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [20]:
grid_minmax.fit(X_train_holdout_mm, y_train_holdout)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [21]:
grid_stan.fit(X_train_holdout_sta, y_train_holdout)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [22]:
grid_stan.best_params_ , grid_minmax.best_params_, grid_search.best_params_

({'n_neighbors': 6, 'weights': 'distance'},
 {'n_neighbors': 7, 'weights': 'distance'},
 {'n_neighbors': 14, 'weights': 'distance'})

In [23]:
grid_stan.best_score_,grid_minmax.best_score_,grid_search.best_score_

(0.5758168875150007, 0.5871001031991744, 0.5991724594505131)

### Treinando com o melhor modelo usando os dados de treino (80%)

In [24]:
knn = KNeighborsClassifier(n_neighbors=14, weights='distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
f1_score(y_test, y_pred)

0.5454545454545454